In [1]:
import pandas as pd
import numpy as np
import re #para identificar patrones en textos
from langdetect import detect #para detectar idiomas, se tiene que instalar
from deep_translator import GoogleTranslator #para traducir texto, se tiene que instalar
import unicodedata #para quitar tildes y reconocer habitación/habitaciones

In [2]:
df1=pd.read_excel('rawdata/v-barcelones-p1.xlsx')
df2=pd.read_excel('rawdata/v-barcelones-p2.xlsx')
df3=pd.read_excel('rawdata/v-barcelones-p3.xlsx')

df=pd.concat([df1, df2, df3], ignore_index=True)

En caso quiera compartir la estructura de `df`.

In [3]:
#df.head().to_dict(orient='records')

# Last modification

In [4]:
# Aplicar la función a la columna solo si el valor es una cadena de texto
df['last_mod'] = df['Last Modification'].apply(lambda x: x.replace('última modificación ', '') if isinstance(x, str) else x)

# Eliminar la columna original
df = df.drop('Last Modification', axis=1)

# Distribution

Obtengo `indice_alquiler`. Son muy pocos los pisos que tienen esta información.

In [5]:
# Función para asignar valores a la nueva columna 'dummy_indice'
def asignar_valor(texto):
    if isinstance(texto, str):  # Verificar si el valor es una cadena de texto
        if re.search(r'Índice Alquiler:\s*(\d+|\D)', texto):
            match = re.search(r'Índice Alquiler:\s*(\d+|\D)', texto)
            if match.group(1).isdigit():
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0

# Aplicar la función a la columna 'Distribution' y crear la nueva columna 'dummy_indice'
df['dummy_indice'] = df['Distribution'].apply(asignar_valor)

In [6]:
# Función para obtener el valor numérico decimal después de "Índice Alquiler"
def obtener_valor(texto):
    if isinstance(texto, str):  # Verificar si el valor es una cadena de texto
        match = re.search(r'Índice Alquiler:\s*([\d,]+)', texto)
        if match:
            valor = match.group(1).replace(',', '.')  # Reemplazar la coma por un punto para obtener un decimal válido
            return float(valor)
        else:
            return 0.0
    else:
        return 0.0

# Aplicar la función para crear la nueva columna 'indice_alquiler'
df['indice_alquiler'] = df['Distribution'].apply(obtener_valor)

# Price other y General Characteristics

## `area`, `n_rooms` y `price_m2`

In [7]:
# Función para extraer la información del precio y otras características
def extraer_info(texto):
    area_match = re.search(r'(\d+)m2', texto)
    area = int(area_match.group(1)) if area_match else 0
    rooms_match = re.search(r'(\d+) habitaciones', texto)
    n_rooms = int(rooms_match.group(1)) if rooms_match else 0
    price_m2_match = re.search(r'(\d+,\d+)€/m2', texto)
    price_m2 = float(price_m2_match.group(1).replace(',', '.')) if price_m2_match else 0
    return pd.Series([area, n_rooms, price_m2])

# Aplicar la función y crear las nuevas columnas
df[['area', 'n_rooms', 'price_m2']] = df['Price_other'].apply(extraer_info)

In [8]:
def extraer_info_description(texto):
    rooms_match = re.search(r'(\d+)\s*(?:habitacion(?:es)?)\b', str(texto))
    n_rooms = int(rooms_match.group(1)) if rooms_match else 0
    return n_rooms

# Aplicar la función extraer_info_description a la columna 'Description' cuando 'n_rooms' sea cero
df['n_rooms'] = df.apply(lambda row: extraer_info_description(row['Description']) if row['n_rooms'] == 0 else row['n_rooms'], axis=1)

In [9]:
def extraer_info_description(texto):
    # Buscar números, palabras numéricas o "una" seguidos de "habitación", "habitaciones" o "habitación" en la descripción
    rooms_match = re.search(r'(?:\b(una|uno|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciséis|diecisiete|dieciocho|diecinueve|veinte)\b|\d+)\s*habitacion(?:es)?\b', str(texto))
    # Convertir palabras numéricas a números
    if rooms_match and rooms_match.group(1):
        num_word = rooms_match.group(1)
        num_dict = {'una': 1, 'uno': 1, 'dos': 2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, 'siete': 7, 'ocho': 8, 'nueve': 9, 'diez': 10,
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quince': 15, 'dieciséis': 16, 'diecisiete': 17, 'dieciocho': 18,
                    'diecinueve': 19, 'veinte': 20}
        n_rooms = num_dict[num_word]
    else:
        # Extraer solo el número encontrado
        n_rooms = int(re.search(r'\d+', rooms_match.group(0)).group()) if rooms_match else 0
    return n_rooms

# Aplicar la función extraer_info_description a la columna 'Description' cuando 'n_rooms' sea cero
df['n_rooms'] = df.apply(lambda row: extraer_info_description(row['Description']) if row['n_rooms'] == 0 else row['n_rooms'], axis=1)

In [10]:
def extraer_info_description2(texto):
    # Buscar "una habitación" en la descripción
    rooms_match = re.search(r'\buna\s*habitación\b', str(texto))

    # Si se encuentra la frase "una habitación", establecer n_rooms en 1, de lo contrario, en 0
    n_rooms = 1 if rooms_match else 0
    
    return n_rooms

# Aplicar la función extraer_info_description a la columna 'Description' cuando 'n_rooms' sea cero
df['n_rooms'] = df.apply(lambda row: extraer_info_description2(row['Description']) if row['n_rooms'] == 0 else row['n_rooms'], axis=1)

## `n_baths`, `terrace`, `n_aseo`, `laundry`, `buhardilla`, `despacho`, `trastero`

In [11]:
# Define expresiones regulares para extraer la información
patron_banos = r'(\d+) Baño'

# Función para extraer el número de baños
def extraer_banos(texto):
    if isinstance(texto, str):  # Verificar si el valor es una cadena de texto
        resultado = re.search(patron_banos, texto)
        if resultado:
            return int(resultado.group(1))
        else:
            return None
    else:
        return None

# Función para extraer la información de las características generales
def extraer_caracteristicas_generales(texto):
    if isinstance(texto, str):  # Verificar si el valor es una cadena de texto
        terrace_match = re.search(r'Terraza (\d+)', texto)
        terrace_m2 = int(terrace_match.group(1)) if terrace_match else 0   

        aseo_match = re.search(r'(\d+) (Aseo|Aseos)', texto)
        n_aseo = int(aseo_match.group(1)) if aseo_match else 0

        laundry = 1 if re.search(r'Lavadero', texto) else 0
        buhardilla = 1 if re.search(r'Buhardilla', texto) else 0
        despacho = 1 if re.search(r'Despacho', texto) else 0
        trastero = 1 if re.search(r'Trastero', texto) else 0

        return pd.Series([terrace_m2, n_aseo, laundry, buhardilla, despacho, trastero])
    else:
        return pd.Series([0, 0, 0, 0, 0, 0])  # Devuelve una serie con valores por defecto si el valor no es una cadena de texto

# Aplicar las funciones y crear las nuevas columnas
df['n_baths'] = df['General Characteristics'].apply(extraer_banos)
# Filtrar filas donde no se encontraron baños en "General Characteristics"
filas_sin_banos = df['n_baths'].isna()
df.loc[filas_sin_banos, 'n_baths'] = df.loc[filas_sin_banos, 'Price_other'].apply(extraer_banos)

# Aplicar la función y crear las nuevas columnas
df[['terrace_m2', 'n_aseo', 'laundry','buhardilla','despacho','trastero']] = df['General Characteristics'].apply(extraer_caracteristicas_generales)

### Mejora de `n_baths`

Buscamos referencia a la cantidad de baños en la descripción.

In [12]:
# Define la función extraer_info_description primero

def extraer_info_description(texto):
    bath_match = re.search(r'\b(\d+|[a-zA-Z]+)\s*(baño|baños)\b', str(texto), re.IGNORECASE)
    if bath_match:
        num_baths = bath_match.group(1)
        if num_baths.isdigit():
            n_baths = int(num_baths)
        else:
            num_dict = {'uno': 1, 'dos': 2, 'tres': 3, 'cuatro': 4, 'cinco': 5}
            n_baths = num_dict.get(num_baths.lower(), 1)
    else:
        n_baths = 0
    return n_baths

# Luego, aplica la función a la columna 'n_baths'

df['n_baths'] = df.apply(lambda row: extraer_info_description(row['Description']) if (pd.isna(row['n_baths']) or row['n_baths'] < 1) else row['n_baths'], axis=1)

In [13]:
def extraer_info_description_bathroom(texto):
    # Buscar la palabra "baño" en la descripción
    bath_match = re.search(r'\b(baño|baño completo)\b', str(texto), re.IGNORECASE)

    # Si se encuentra la palabra "baño"
    if bath_match:
        return 1  # Si encuentra la palabra "baño", establecemos n_baths en 1
    else:
        return 0  # Si no se encuentra la palabra "baño", establecemos n_baths en 0

# Aplicar la función extraer_info_description a las celdas con valores menores que 1 o nulos

df['n_baths'] = df.apply(lambda row: extraer_info_description_bathroom(row['Description']) if (pd.isna(row['n_baths']) or row['n_baths'] < 1) else row['n_baths'], axis=1)

Los `n_baths` que tienen valor 0 es porque olvidaron cargar información al respecto. Al revisar las fotos veo que sí tienen baño. <mark> Importante considerar para hacer limpieza del df</mark>.

### Mejora de `terrace_m2`

Obtengo una variable dicotómica (`terrace`), dado que en la descripción algunos pisos indican tener terraza, pero no indican con exactitud sus m$^2$. Por si las dudas también mejoro `terrace_m2` con información de la descripción.

*Tengo una columna llamada 'terrace_m2'. Cuando sus valores sean ceros quiero hacer lo siguiente: ir a la columna 'Description' y buscar la palabra 'terraza', 'Terraza' o sus equivalentes en plural y continuación ver si en una distancia de máximo dos palabras hacia atrás o hacia adelante hay un número. Si lo hay, colocarlo en 'terrace_m2', si no indicar el valor 0 en esa columna.*

In [14]:
def buscar_numero(texto):
    if isinstance(texto, str):  # Verificar si texto es una cadena de texto
        # Buscar la palabra 'terraza' o 'Terraza' y un número en un rango de dos palabras
        match = re.search(r'(\b(?:\w+\s+){0,2}\d+\s+(?:terraza|Terraza)\b|\b(?:\w+\s+){0,2}(?:terraza|Terraza)\s+\d+)', texto)
        if match:
            # Extraer el número encontrado
            numero = re.search(r'\d+', match.group()).group()
            return int(numero)
    return 0

# Iterar sobre cada fila
for index, row in df.iterrows():
    # Verificar si el valor en 'terrace_m2' es cero
    if row['terrace_m2'] == 0:
        # Buscar la palabra 'terraza' o 'Terraza' y un número en 'Description'
        numero_terrace = buscar_numero(row['Description'])
        # Actualizar 'terrace_m2' con el número encontrado o 0 si no se encuentra
        df.at[index, 'terrace_m2'] = numero_terrace

*Ahora vamos a crear una nueva variable que será dicotómica: 'terrace'. Cuando el valor de terrace_m2 sea mayor a 1, 'terrace' tendrá el valor 1; cuando no, 0. Una vez hecho eso, únicamente sobre los valores 0 añadiremos otra regla: cuando en la columna Description se haga mención a la palabra 'terraza', 'Terraza' o sus equivalentes en plural, cambiar el valor de 'terrace' por 1.*

In [15]:
# Paso 1: Crear la variable dicotómica 'terrace' basada en 'terrace_m2'
df['terrace'] = (df['terrace_m2'] > 1).astype(int)

# Paso 2: Aplicar la regla adicional para actualizar 'terrace' cuando 'terrace_m2' sea igual a 0
for index, row in df.iterrows():
    if row['terrace_m2'] == 0 and isinstance(row['Description'], str):  # Verificar si 'Description' es una cadena de texto
        # Buscar la palabra 'terraza' o 'Terraza' en 'Description' (ignorando mayúsculas y minúsculas)
        if re.search(r'(?:\bterraza|Terraza)s?(?:\b|\w{0,10}\b)', row['Description'], re.IGNORECASE): # Ajuste para incluir errores de espaciado
            # Actualizar 'terrace' a 1 si se encuentra una mención a la terraza
            df.at[index, 'terrace'] = 1

Hay casos en los que el texto de la descripción tiene problemas de redacción y el código no funciona bien:

- Ejemplo 1: "(...)2 Baño Balcón-TerrazaParqueoSituado en un lugar(...)"
- Ejemplo 2: "(...)hay una salida directa a una amplia terrazaEn la zona de noche(...)"

Sin embargo, son casos aislados. En el caso del municipio de Barcelona, de un total de 3424 casos, sólo 5 no pudieron ser "captados" por el código.

# EPC

## `EPC_Consum`

In [16]:
# Convertir 'EPC_Consum' column a strings
df['EPC_Consum'] = df['EPC_Consum'].astype(str)

# Función para extraer la letra de la segunda línea
def extraer_letra(texto):
    resultado = re.search(r'Consumo:\n+\s+(\w)', texto)
    if resultado:
        return resultado.group(1)
    else:
        return None

# Función para extraer el valor numérico de la tercera línea
def extraer_valor(texto):
    resultado = re.search(r'E\n+\s+(\d+)', texto)
    if resultado:
        return int(resultado.group(1))
    else:
        return None

# Aplicar las funciones y crear las nuevas columnas
df['consum_EPC'] = df['EPC_Consum'].apply(extraer_letra)
df['kwhm2_year'] = df['EPC_Consum'].apply(extraer_valor)

## `EPC_Emission`

In [17]:
# Convertir las letras a variables dummy en EPC_Emission
def extraer_letra_emission(texto):
    resultado = re.search(r'Emisiones: (\w)', texto)
    if resultado:
        return resultado.group(1)
    else:
        return None

# Función para extraer el valor numérico de la tercera línea en EPC_Emission
def extraer_valor_emission(texto):
    resultado = re.search(r'(\d+) kg CO2 m2 / año', texto)
    if resultado:
        return int(resultado.group(1))
    else:
        return None

# Aplicar las funciones y crear las nuevas columnas
df['emission_EPC'] = df['EPC_Emission'].astype(str)
df['emission_EPC'] = df['emission_EPC'].apply(extraer_letra_emission)
df['kgCO2m2_year'] = df['EPC_Emission'].astype(str)
df['kgCO2m2_year'] = df['kgCO2m2_year'].apply(extraer_valor_emission)

### `dummy_consum_EPC` y `dummy_emission_EPC`

In [18]:
# Función para crear las columnas dummy_consum_EPC y dummy_emission_EPC
def crear_dummy(valor):
    if pd.notna(valor):
        return 1
    else:
        return 0

# Aplicar la función y crear las nuevas columnas
df['dummy_consum_EPC'] = df['consum_EPC'].apply(crear_dummy)
df['dummy_emission_EPC'] = df['emission_EPC'].apply(crear_dummy)

### One-hot encoding

In [19]:
# Convertir las letras a variables dummy
consum_dummy = pd.get_dummies(df['consum_EPC'], prefix='consum', dummy_na=False, dtype=int)
emission_dummy = pd.get_dummies(df['emission_EPC'], prefix='emission', dummy_na=False, dtype=int)

# Concatenar las variables dummy al dataframe original
df = pd.concat([df, consum_dummy, emission_dummy], axis=1)

# Equipment

Obtengo `AC`, `parking`, `pool`, `lift`, `furniture`, `public_transp` (que es algo que ahora los ofertantes pueden publicitar) y `year_cons`(son pocos los pisos que tienen esta info, yo  <mark> consideraría quitarla luego</mark>).

In [20]:
# Función para extraer la información de las características generales
def extraer_caracteristicas(texto):
    if isinstance(texto, str):  # Verificamos si el valor es una cadena
        ac = 1 if re.search(r'Aire acondicionado(\s*:\s*Sí)?', texto) else 0
        parking = 1 if re.search(r'Plaza parking', texto) else 0
        pool = 1 if re.search(r'Piscina comunitaria', texto) else 0
        lift = 1 if re.search(r'Ascensor', texto) else 0
        furniture = 1 if re.search(r'Amueblado', texto) else 0
        public_transp = 1 if re.search(r'Cerca de transporte público', texto) else 0
        year_construc_match = re.search(r'Año construcción (\d{4})', texto)
        year_construc = int(year_construc_match.group(1)) if year_construc_match else 0
    else:
        ac, parking, pool, lift, furniture, public_transp, year_construc = 0, 0, 0, 0, 0, 0, 0
    return pd.Series([ac, parking, pool, lift, furniture, public_transp, year_construc])

# Aplicar la función y crear las nuevas columnas
df[['AC', 'parking', 'pool', 'lift', 'furniture', 'public_transp', 'year_cons']] = df['Equipment'].apply(extraer_caracteristicas)

# Floor

También son pocos los pisos que declaran en qué planta se encuentran, <mark>consideraría sacarla de la base de datos</mark>.

In [21]:
# Función para aplicar las reglas y crear la nueva columna 'floor'
def assign_floor(row):
    if isinstance(row['Equipment'], str) and re.search(r'Planta número (\d+)', row['Equipment']):
        return int(re.search(r'Planta número (\d+)', row['Equipment']).group(1))
    elif isinstance(row['Equipment'], str) and re.search(r'Planta baja', row['Equipment']):
        return 0
    else:
        return 999

# Apply the function to each row of the dataframe
df['floor'] = df.apply(assign_floor, axis=1)

# Drop the original column
df = df.drop('Equipment', axis=1)

# Adicionales

## `barrio`

In [22]:
# Dividir el texto por el guión y seleccionar la segunda parte
df['barrio'] = df['Location'].str.split(' - ', expand=True)[1]

df=df.drop('Location',axis=1)

## `precio_euros`

In [23]:
# Convertir los valores de la columna 'precio_euros' en cadenas de texto
df['precio_euros'] = df['precio_euros'].astype(str)

# Eliminar el punto y el símbolo del euro de cada cadena
df['precio_euros'] = df['precio_euros'].str.replace('.', '').str.replace('€', '')

# Convertir las cadenas resultantes en números enteros o flotantes, convirtiendo los "nan" en 0
df['precio_euros'] = pd.to_numeric(df['precio_euros'], errors='coerce').fillna(0)

## `codigo_inmueble`

In [24]:
# Definir una función para extraer el número de la cadena
def extraer_numero(texto):
    if isinstance(texto, str):  # Verificar si el valor es una cadena de texto
        resultado = re.search(r'\d+', texto)
        if resultado:
            return int(resultado.group())
        else:
            return None
    else:
        return None

# Aplicar la función a la columna 'codigo_inmueble' y crear una nueva columna 'codigo_inmueble_numero'
df['codigo_inmueble_numero'] = df['codigo_inmueble'].apply(extraer_numero)

## `Lon_X` y `Lat_Y`

In [25]:
# Función para limpiar y extraer la latitud
def extraer_latitud(cadena):
    partes = cadena.split(',')
    longitud = partes[0].strip()  # Obtenemos la longitud
    # Limpiamos y extraemos la latitud
    latitud_raw = partes[1].replace('\\"VGPSLon\\":', '').strip()
    latitud = latitud_raw.rstrip(',')  # Limpiamos cualquier carácter adicional al final
    return longitud, latitud

# Aplicamos la función y creamos las nuevas columnas 'Longitud' y 'Latitud'
df[['Lat_Y', 'Lon_X']] = df['Lon/Lat'].apply(lambda x: pd.Series(extraer_latitud(x)))

Lo que veo al revisar `df` hasta el paso anterior es que hay dos casos en los que la latitud y longitud no se extrayeron correctamente. En un caso tienen el código html de la web y en otro valores 0.000 para la longitud y la latitud. Los elimino a continuación.

In [26]:
# Primero, limpiemos las celdas que contienen valores no numéricos
df['Lon_X'] = pd.to_numeric(df['Lon_X'], errors='coerce')
df['Lat_Y'] = pd.to_numeric(df['Lat_Y'], errors='coerce')

# Luego, eliminamos las filas que contienen valores no numéricos
df = df.dropna(subset=['Lon_X', 'Lat_Y'])

# Finalmente, convertimos las columnas a números flotantes
df['Lon_X'] = df['Lon_X'].astype(float)
df['Lat_Y'] = df['Lat_Y'].astype(float)

## `multifam`

In [27]:
# Buscar las palabras "casa", "chalet" o "masía" en las columnas especificadas
mask = df['Title'].str.contains(r'\b(?:casa|chalet|masía)\b', flags=re.IGNORECASE) | \
       df['Description'].str.contains(r'\b(?:casa|chalet|masía)\b', flags=re.IGNORECASE) | \
       df['texto_destacado'].str.contains(r'\b(?:casa|chalet|masía)\b', flags=re.IGNORECASE)

# Crear la columna 'piso' con los valores correspondientes (1 y 0)
df['multifam'] = ~mask

# Convertir los valores booleanos a 1 y 0
df['multifam'] = df['multifam'].astype(int)

In [28]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['Price_other', 'texto_destacado', 'Distribution', 'General Characteristics', 'EPC_Consum', 'EPC_Emission', 'Lon/Lat']

# Eliminar las columnas
df = df.drop(columnas_a_eliminar, axis=1)

In [29]:
df.to_excel('data/pre-procesado/v-barcelones.xlsx', index=False)